In [30]:
import pandas as pd
import numpy as np

In [31]:
df = pd.read_csv('melb_data_fe.csv', sep=',')
df.sample(5)

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
8665,other,3,townhouse,815000.0,S,Williams,2017-05-13,9.1,3015,3,...,144.87050,Western Metropolitan,984,15.75,-0.080292,5,47,5,St,1
10339,Heidelberg West,2,house,532000.0,S,Ray,2017-05-27,8.8,3081,2,...,145.04870,Eastern Metropolitan,2674,21.00,-0.271676,5,47,5,Rd,1
8516,Williamstown,2,unit,550000.0,SP,Williams,2017-04-22,8.0,3016,2,...,144.90580,Western Metropolitan,6380,13.00,1.000000,4,27,5,St,1
12624,other,3,house,616000.0,S,Barry,2017-09-09,16.5,3049,3,...,144.90220,Northern Metropolitan,2474,18.00,-0.687732,9,47,5,Ct,1
9542,Fitzroy North,2,house,1270000.0,S,Nelson,2017-06-17,3.6,3068,2,...,144.97991,Northern Metropolitan,6244,25.20,-0.184466,6,127,5,St,1


In [32]:
df['Date'] = pd.to_datetime(df['Date'])
quarter = df.Date.dt.quarter
(quarter == 2).shape[0]

13580

In [33]:
def transform_to_category(df: pd.DataFrame, exclude=['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']):
    for column in df.columns:
        if df[column].value_counts().shape[0] < 150 and column not in exclude:
            print(f'Converting column {column}')
            df[column] = df[column].astype('category')

transform_to_category(df)

Converting column Suburb
Converting column Type
Converting column Method
Converting column SellerG
Converting column CouncilArea
Converting column Regionname
Converting column MonthSale
Converting column WeekdaySale
Converting column StreetType
Converting column Weekend


In [34]:
(df.dtypes == 'category').value_counts()
df.dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

In [35]:
df.sort_values(by='AreaRatio', ignore_index=True, ascending=False).iloc[1558]['BuildingArea']

126.0

In [36]:
mask1 = df.Type == 'townhouse'
mask2 = df.Rooms > 2
df_copy = df[mask1 & mask2]
df_copy.sort_values(by=['Rooms', 'MeanRoomsSquare'], ascending=[True, False], ignore_index=True).iloc[18]

Suburb                     Brighton East
Rooms                                  3
Type                           townhouse
Price                          1300000.0
Method                                PI
SellerG                           Buxton
Date                 2016-09-17 00:00:00
Distance                            10.7
Postcode                            3187
Bedroom                                3
Bathroom                               2
Car                                    2
Landsize                           337.0
BuildingArea                       222.0
CouncilArea                      Bayside
Lattitude                       -37.9196
Longtitude                      145.0205
Regionname         Southern Metropolitan
Propertycount                       6938
MeanRoomsSquare                    27.75
AreaRatio                      -0.205725
MonthSale                              9
AgeBuilding                           16
WeekdaySale                            5
StreetType      

In [37]:
df.groupby(by='Rooms')['Price'].mean()

Rooms
1     4.338245e+05
2     7.750812e+05
3     1.076081e+06
4     1.445282e+06
5     1.870260e+06
6     1.849366e+06
7     1.920700e+06
8     1.602750e+06
10    9.000000e+05
Name: Price, dtype: float64

In [38]:
df.groupby(by='Regionname')['Lattitude'].std()

Regionname
Eastern Metropolitan          0.047890
Eastern Victoria              0.147067
Northern Metropolitan         0.049639
Northern Victoria             0.084455
South-Eastern Metropolitan    0.073411
Southern Metropolitan         0.043080
Western Metropolitan          0.051251
Western Victoria              0.011579
Name: Lattitude, dtype: float64

In [39]:
mask = pd.to_datetime(df.Date) > '2017-05-01'
mask2 = pd.to_datetime(df.Date) < '2017-09-01'
display(df[mask & mask2])
df[mask & mask2].groupby(by='SellerG')['Price'].sum().sort_values()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
7431,Aberfeldie,4,house,1330000.0,S,Nelson,2017-06-05,9.1,3040,4,...,144.89590,Western Metropolitan,1543,12.600000,-0.368421,6,47,0,St,0
7432,Aberfeldie,3,house,1300000.0,S,Nelson,2017-08-04,9.1,3040,3,...,144.89160,Western Metropolitan,1543,26.666667,0.000000,8,4,4,St,0
7433,Aberfeldie,2,house,470000.0,S,Barry,2017-08-04,9.1,3040,2,...,144.90430,Western Metropolitan,1543,14.460000,0.016163,8,37,4,St,0
7435,Albert Park,3,house,1700000.0,VB,Greg,2017-06-05,3.3,3206,3,...,144.94950,Southern Metropolitan,3280,16.571429,-0.210884,6,107,0,St,0
7436,Albert Park,1,unit,1100000.0,PI,Nelson,2017-08-04,3.3,3206,1,...,144.95110,Southern Metropolitan,3280,21.333333,1.000000,8,129,4,Pde,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,other,4,house,1245000.0,S,Barry,2017-08-26,16.7,3150,4,...,145.16761,South-Eastern Metropolitan,7392,12.600000,-0.676093,8,36,5,Cr,1
13576,Williamstown,3,house,1031000.0,SP,Williams,2017-08-26,6.8,3016,3,...,144.87904,Western Metropolitan,6380,16.625000,-0.429185,8,22,5,Dr,1
13577,Williamstown,3,house,1170000.0,S,Raine,2017-08-26,6.8,3016,3,...,144.88738,Western Metropolitan,6380,15.750000,-0.551601,8,20,5,St,1
13578,Williamstown,4,house,2500000.0,PI,Sweeney,2017-08-26,6.8,3016,4,...,144.89299,Western Metropolitan,6380,17.444444,-0.693060,8,97,5,St,1


SellerG
LITTLE             2742000.0
Cayzer             4439000.0
Burnham            4550500.0
Moonee             7328000.0
Thomson            8332000.0
Bells              8656000.0
Alexkarbon        10985000.0
McDonald          14637500.0
Rendina           15422276.0
Nick              16890000.0
Douglas           18341000.0
Buckingham        19033000.0
C21               19515000.0
Eview             19791500.0
Collins           20217000.0
Philip            22051800.0
Chisholm          23225000.0
Williams          23297000.0
Love              23365500.0
Purplebricks      23401000.0
O'Brien           23855508.0
HAR               25568000.0
Village           26473000.0
RW                29261000.0
Raine             30687700.0
Stockdale         35409800.0
Sweeney           36882750.0
Gary              39138400.0
Hodges            43231000.0
YPA               46354350.0
Miles             47582000.0
Kay               48569500.0
RT                50498000.0
Brad              55955000.0
Jas   

In [40]:
df.pivot_table(values='Price', index='SellerG', columns='Type', aggfunc='median')['unit'].sort_values()

SellerG
Burnham          305000.0
Sweeney          381000.0
Stockdale        410000.0
Village          420000.0
Love             440000.0
YPA              443500.0
LITTLE           447000.0
Raine            447500.0
Jas              455000.0
Bells            457500.0
Rendina          459000.0
Brad             465000.0
Douglas          494500.0
Thomson          495000.0
Ray              497500.0
Harcourts        500000.0
Barry            502000.0
McDonald         502500.0
HAR              518000.0
Nelson           530000.0
other            535000.0
McGrath          542500.0
Greg             547500.0
Biggin           548000.0
Gary             550000.0
Buckingham       553000.0
Miles            555000.0
Alexkarbon       560000.0
hockingstuart    566000.0
Williams         577000.0
Purplebricks     582500.0
Jellis           591750.0
Collins          592000.0
Woodards         600000.0
Eview            602000.0
Hodges           605500.0
O'Brien          618500.0
Moonee           622000.0
RW  

In [41]:
### Joins

raitings1 = pd.read_csv('ratings1.csv')
raitings2 = pd.read_csv('ratings2.csv')
movies = pd.read_csv('movies.csv')
dates = pd.read_csv('dates.csv')

In [42]:
movies['movieId'].value_counts().shape
raitings1['userId'].value_counts().shape

(274,)

In [43]:
dates.date = pd.to_datetime(dates['date'])
dates.date.dt.year.value_counts()

2000    10061
2017     8198
2007     7114
2016     6703
2015     6616
2018     6418
1996     6040
2005     5813
2012     4656
2008     4351
2009     4158
2006     4059
2003     4014
2001     3922
2002     3478
2004     3279
1999     2439
2010     2301
1997     1916
2011     1690
2013     1664
2014     1439
1998      507
Name: date, dtype: int64

In [44]:
ratings = pd.concat([raitings1, raitings2], ignore_index=True)
ratings.drop_duplicates(ignore_index=True, inplace=True)

ratings_dates = pd.concat([ratings, dates], axis=1)

ratings_dates_movies = ratings_dates.merge(movies, how='left', on='movieId')

In [45]:
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None


In [46]:
ratings_dates_movies['year_release'] = ratings_dates_movies.title.apply(get_year_release)

In [47]:
ratings_dates_movies.year_release.info()

<class 'pandas.core.series.Series'>
Int64Index: 100836 entries, 0 to 100835
Series name: year_release
Non-Null Count   Dtype  
--------------   -----  
100818 non-null  float64
dtypes: float64(1)
memory usage: 1.5 MB


In [48]:
mask = ratings_dates_movies.year_release == 1999

ratings_dates_movies[mask].groupby(by='movieId')['rating'].mean().sort_values()
mask2 = ratings_dates_movies.movieId == 145951
ratings_dates_movies[mask2]

,userId,movieId,rating,date,title,genres,year_release
100797,610,145951,0.5,2017-05-03 22:03:39,Bloodsport: The Dark Kumite (1999),Action|Thriller,1999.0


In [49]:
sample = ratings_dates_movies[ratings_dates_movies.year_release == 2010]
sample.groupby(by='genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [50]:
ratings_dates_movies.groupby(by='userId')['genres'].nunique().sort_values()

userId
214     13
85      13
245     13
494     15
578     15
      ... 
474    395
380    399
448    403
414    482
599    524
Name: genres, Length: 610, dtype: int64

In [51]:
sample = ratings_dates_movies.groupby(by='userId')['rating'].agg(['count', 'mean'])
sample.sort_values(by=['count', 'mean'], ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [52]:
filtered = ratings_dates_movies[ratings_dates_movies['year_release'] == 2018]
filtered = filtered.groupby(by='genres')['rating'].agg(['mean', 'count'])
filtered[filtered['count'] > 10]

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [53]:
ratings_dates_movies['year_rating'] = pd.to_datetime(ratings_dates_movies.date).dt.year
ratings_dates_movies['year_rating']

ratings_dates_movies.pivot_table(
    values='rating',
    index='genres',
    columns='year_rating',
    aggfunc='mean'
).loc['Animation|Children|Mystery']



year_rating
1996    NaN
1997    NaN
1998    NaN
1999    NaN
2000    NaN
2001    NaN
2002    NaN
2003    NaN
2004    NaN
2005    NaN
2006    NaN
2007    NaN
2008    NaN
2009    NaN
2010    NaN
2011    NaN
2012    NaN
2013    NaN
2014    NaN
2015    NaN
2016    NaN
2017    NaN
2018    5.0
Name: Animation|Children|Mystery, dtype: float64

In [54]:
orders = pd.read_csv('orders.csv', sep=';')
products = pd.read_csv('products.csv', sep=';')
display(orders.shape)
display(products.head())

(18, 9)

,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR


In [55]:
orders_products = orders.merge(products, how='left', left_on='ID товара', right_on='Product_ID')
orders_products[pd.isna(orders_products['Product_ID'])]

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
17,01.01.2001 00:00:00,0,1,"Оплачен, формируется к отправке",Да,Нет,Нет,666,1,NaN,NaN,NaN,NaN


In [56]:
orders_products[orders_products['Отменен'] == 'Да']

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,124.0,Носки беговые Camino,999.0,RUR


In [57]:
orders_products['total_sum'] = orders_products['Количество'] * orders_products['Price']
orders_products.groupby(by='ID Покупателя')['total_sum'].sum()

ID Покупателя
1      2240.0
5     14042.0
7     17096.0
8      3087.0
9      7040.0
10      995.0
Name: total_sum, dtype: float64

# SEABORN